In [1]:
import pandas as pd
import keras_nlp
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn

from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Input
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Embedding

In [2]:
cards = pd.read_json("data/cards.json")
cards

,name,rules_text,colors,color_identity,flavour_text,type_line,power,toughness,set
0,Static Orb,"As long as CARDNAME is untapped, players can't...","[0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 1]",NaN,Artifact,None,None,7ed
1,Sensory Deprivation,Enchant creature\nEnchanted creature gets -3/-0.,"[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",NaN,Enchantment — Aura,None,None,m14
2,Road of Return,Choose one —\n• Return target permanent card f...,"[0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 1, 0]",NaN,Sorcery,None,None,c19
3,Storm Crow,Flying (This creature can't be blocked except ...,"[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",NaN,Creature — Bird,1,2,9ed
4,Walking Sponge,tap: Target creature loses your choice of fly...,"[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",NaN,Creature — Sponge,1,1,ulg
...,...,...,...,...,...,...,...,...,...
24444,Quarry Beetle,"When CARDNAME enters the battlefield, you may ...","[0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 1, 0]",NaN,Creature — Insect,4,5,hou
24445,Devoted Hero,,"[1, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]",NaN,Creature — Elf Soldier,1,2,s99
24446,Without Weakness,Target creature you control gains indestructib...,"[0, 0, 1, 0, 0, 0]","[0, 0, 1, 0, 0, 0]",NaN,Instant,None,None,hou
24447,Firesong and Sunspeaker,Red instant and sorcery spells you control hav...,"[1, 0, 0, 1, 0, 0]","[1, 0, 0, 1, 0, 0]",NaN,Legendary Creature — Minotaur Cleric,4,6,2x2


In [3]:
X = cards["rules_text"]
Y = cards["color_identity"]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

In [4]:
nr_of_targets= 5
y_train = [np.asarray(y)[0:nr_of_targets].astype('float32').ravel() for y in y_train] 
y_test = [np.asarray(y)[0:nr_of_targets].astype('float32').ravel() for y in y_test]
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [5]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)

In [6]:
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [7]:
vocab_size = len(tokenizer.word_index) + 2
maxlen = 50

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [9]:
from numpy import array
from numpy import asarray
from numpy import zeros

#https://keras.io/examples/nlp/pretrained_word_embeddings/
embeddings_index = {}
with open('data/glove.6B.100d.txt', encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Found 400000 word vectors.
Converted 2222 words (505 misses)


In [10]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(nr_of_targets, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [11]:
history = model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/20
123/123 [==============================] - 9s 39ms/step - loss: 0.5270 - acc: 0.2339 - val_loss: 0.5157 - val_acc: 0.1659
Epoch 2/20
123/123 [==============================] - 4s 33ms/step - loss: 0.5152 - acc: 0.2176 - val_loss: 0.5144 - val_acc: 0.1664
Epoch 3/20
123/123 [==============================] - 4s 33ms/step - loss: 0.5135 - acc: 0.2411 - val_loss: 0.5184 - val_acc: 0.3211
Epoch 4/20
123/123 [==============================] - 4s 33ms/step - loss: 0.5018 - acc: 0.2700 - val_loss: 0.4877 - val_acc: 0.3226
Epoch 5/20
123/123 [==============================] - 4s 33ms/step - loss: 0.4569 - acc: 0.3656 - val_loss: 0.4344 - val_acc: 0.4128
Epoch 6/20
123/123 [==============================] - 4s 33ms/step - loss: 0.4182 - acc: 0.4535 - val_loss: 0.4017 - val_acc: 0.4645
Epoch 7/20
123/123 [==============================] - 4s 33ms/step - loss: 0.3819 - acc: 0.5274 - val_loss: 0.3847 - val_acc: 0.5003
Epoch 8/20
123/123 [==============================] - 4s 33ms/step - 

In [12]:
score = model.evaluate(x_test, y_test, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

153/153 [==============================] - 1s 8ms/step - loss: 0.3145 - acc: 0.6284
Test Score: 0.314476877450943
Test Accuracy: 0.6284253597259521


In [13]:
predictions = model.predict(x_test)

153/153 [==============================] - 1s 5ms/step


In [14]:
predictions

array([[0.04808267, 0.03192785, 0.6846058 , 0.13833001, 0.1567531 ],
       [0.00632542, 0.01756191, 0.04748635, 0.73328656, 0.01997124],
       [0.00294442, 0.41260827, 0.82829916, 0.00405894, 0.00867659],
       ...,
       [0.57084244, 0.17063533, 0.05390633, 0.01598805, 0.12810197],
       [0.02420046, 0.96644586, 0.02234758, 0.01348739, 0.01151406],
       [0.02905421, 0.0390164 , 0.01169936, 0.25367045, 0.5904845 ]],
      dtype=float32)

In [15]:
y_test

<tf.Tensor: shape=(4890, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)>